Example notebook

In [1]:
import subprocess
import re

r_cell_re = re.compile(r'^```\{r[^\}]+\}')

#def number_of_rmarkdown_cells(rmd_file: str) -> int:
rmd_file = '../demo/notebooks/example_rmarkdown.Rmd'

def number_of_r_cells_in_rmd(rmd_file: str) -> int:
	number_of_r_cells = 0
	with open(rmd_file) as rmd:
		for rmd_line in rmd:
			if r_cell_re.match(rmd_line):
				number_of_r_cells += 1
	return number_of_r_cells

In [5]:
x = '  |................................................................................                                                            |  57%'
x


'  |................................................................................                                                            |  57%'

In [22]:
percent_re = re.compile(r'^.*\|[\s]+([0-9]+)%.*$')

In [24]:
int(percent_re.sub(r'\1', x))

57

In [44]:
x

'  |................................................................................                                                            |  57%'

In [53]:
start_re = re.compile(r'^[\s]*\|[\.\s]*\|[\s]*([0-9]+)%$')

In [49]:
start_re.match(x)

<re.Match object; span=(0, 2), match='  '>

In [54]:
start_re.sub(r'\1', x)

'57'

In [57]:
from tqdm import tqdm
import sys
from bluprint.colors import Style

In [69]:
def tqdm_format() -> str:
    sep = f'{Style.cyan}─{Style.end}'
    return (
        '{desc} {percentage:3.0f}% '
        + sep + ' Elapsed: {elapsed}'
    )

def execute_rmd(rmd_file: str) -> None:
	progress_bar_re = re.compile(r'^[\s]*\|[\.\s]*\|[\s]*([0-9]+)%$')
	with tqdm(total=100, bar_format=tqdm_format()) as tqdm_progress:
		rscript_cmd = f'rmarkdown::render("{rmd_file}")'
		rmd_out = subprocess.Popen(
			['Rscript', '-e', rscript_cmd],
			bufsize=1,
			# cwd='',
			universal_newlines=True,
			stdout=subprocess.PIPE,
			stderr=subprocess.PIPE,
		)
		last_percent = 0
		for line in rmd_out.stdout:
			if progress_bar_re.match(line):
				current_percent = int(progress_bar_re.sub(r'\1', line))
				tqdm_progress.update(current_percent - last_percent)
				# print(f'pct: {current_percent}')
				sys.stdout.flush()
				last_percent = current_percent
		rmd_out.stdout.close()

In [70]:
execute_rmd('../demo/notebooks/example_rmarkdown.Rmd')

 100% ─ Elapsed: 00:01
